In [1]:
import pandas as pd

In [2]:
affi = pd.read_csv("../data/authors_affiliations.csv")

In [3]:
affi["Author Name"] = [str(i).lower() for i in affi["Author Name"]]
affi["Affiliation"] = [str(i).lower() for i in affi["Affiliation"]]

In [4]:
def extract(row):
    entries = row.split(";")
    check = ["department", "institution", "address"]
    dic = {}
    for i in check:
        lst = []
        for entry in entries:
            if i + ":" in entry:
                lst.append(entry.split(":")[1])
        dic[i] = lst
    return dic          

In [5]:
ex = affi["Affiliation"].apply(lambda x: extract(x))

In [6]:
affi["department"] = [i["department"] for i in ex]
affi["institution"] = [i["institution"] for i in ex]
affi["address"] = [i["address"] for i in ex]

In [7]:
check_list = ["microsoft", "research", "meta", "google", "zurich", "nvidia", "university", "cmu", "eleutherai", "leuven", "madison", "caltech", "cuhk", "technology", "university", "nyu", "mit", "ucla", "berlin", "kaust", "ibm", "apple", "sony", "tum", "openai", "qcraft", "ethz", "cas", "cnic", "tencent"]
for idx, row in affi.iterrows():
    i = row["Author Name"]
    for institution in check_list:
        if institution in i:
            affi.loc[idx, "institution_ap"] = [i]

In [8]:
def isNaN(num):
    return num != num

In [9]:
affi["institution_merge"] = [a.append(b) if not isNaN(b) else a for a,b in zip(affi["institution"], affi["institution_ap"]) ]

In [18]:
from collections import Counter
def merge_(row):
    row = row.fillna("")
    return Counter([j for i in row.tolist() for j in i]).keys()

In [19]:
agg_affi =  affi.fillna("").groupby("arXiv ID", as_index=False).institution_merge.apply(lambda x: merge_(x))
agg_affi["n_institution"] = [len(i) for i in agg_affi["institution_merge"]]

In [20]:
agg_affi

,arXiv ID,institution_merge,n_institution
0,arxiv:2301.00493,(),0
1,arxiv:2301.00497,(),0
2,arxiv:2301.00514,( huazhong university of science and technolog...,5
3,arxiv:2301.00519,(),0
4,arxiv:2301.00537,"( university of michigan, columbia university)",2
...,...,...,...
9575,arxiv:2310.17437,"( universidad nacional de la plata, instituto...",2
9576,arxiv:2310.17458,"( university of cambridge, tsinghua university)",2
9577,arxiv:2310.17502,( university of stuttgart),1
9578,arxiv:2310.17596,( the university of texas at austin),1


In [22]:
# step1 affiliation
affi_df = pd.DataFrame()
for idx, row in agg_affi.sort_values("n_institution", ascending = False).iterrows():
    tmp = pd.DataFrame({"id":[row["arXiv ID"]]*len(row["institution_merge"]),"institution":row["institution_merge"]})
    affi_df = pd.concat([affi_df, tmp])

In [24]:
affi_df["institution"] = [i[1:] for i in affi_df.institution]
affi_df.to_csv("affiliation.csv", index= False)

In [25]:
affi_df

,id,institution
0,arxiv:2302.01790,hi helmholtz imaging
1,arxiv:2302.01790,heidelberg university
2,arxiv:2302.01790,nct heidelberg
3,arxiv:2302.01790,michael baumgartner
4,arxiv:2302.01790,emre kavur
...,...,...
0,arxiv:2305.16947,university of edinburgh
0,arxiv:2305.18189,stanford university
0,arxiv:2305.16965,tsinghua university
0,arxiv:2305.17529,university of central florida ‡ adobe research...


In [32]:
df = pd.DataFrame(Counter(affi_df.institution.tolist()), index = [0]).T.reset_index()
df.columns = ["institution", "count"]

In [33]:
df = df.sort_values("count", ascending = False)

In [35]:
import json
with open('/Users/zhangran/Documents/GitHub/Quaterly-Arxiv/data/location.json', 'rb') as fp:
    m = json.load(fp)

In [42]:
lst = []
for i,v in m.items():
    v = v.replace("('", "").replace("[", "").replace(")", "").replace("]", "").replace("'", "").replace(" ('", "").replace("\"", "")
    if "\n " in v:
        splits = v.replace("- ", "").split("\n ")
    else:
        splits = v.replace("- ", "").split("\n")
    for i in splits:
        lst.append(i)

In [53]:
ins_loc = pd.DataFrame(lst, columns = ["backup"])

In [60]:
ins_loc["location"] = [i.split(",")[1].lower() if len(i.split(","))>1 else i for i in ins_loc.backup]
ins_loc["institution"] = [i.split(",")[0].lower() for i in ins_loc.backup]

In [63]:
ins_loc.to_csv("institution_loc.csv", index = False)

In [68]:
uni = pd.read_csv("../code/world-universities.csv", header = None)
uni.columns = ["country", "university", "link"]

In [74]:
uni["abbre"] = [i.split(".")[1] for i in uni["link"]]

In [75]:
uni.to_csv("../code/world-universities.csv", index = False)

In [81]:
import openai
openai.api_key = ""
def ChatGPT_request(prompt_user):

      """
      Given a prompt and a dictionary of GPT parameters, make a request to OpenAI
      server and returns the response. 
      ARGS:
        prompt: a str prompt
        gpt_parameter: a python dictionary with the keys indicating the names of  
                       the parameter and the values indicating the parameter 
                       values.   
      RETURNS: 
        a str of GPT-3's response. 
      """
      # temp_sleep()
      try: 
        completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106", 
        messages=[{"role": "user", "content": prompt_user},
        #{"role": "system", "content": prompt_sys}
                 ],
        temperature=1.2,
        max_tokens = 3000,
        )
        return completion["choices"][0]["message"]["content"]

      except:
        completion = openai.ChatCompletion.create(
              model="gpt-3.5-turbo-1106",
              messages=[{"role": "user", "content": prompt_user},
                        #{"role": "system", "content": prompt_sys}
                       ],
              temperature=1.2,
            max_tokens = 3000,
          )
        print ("ChatGPT ERROR")
        return "ChatGPT ERROR"

In [87]:
unkown = pd.read_csv("../data/affiliation - unknow.csv", chunksize = 50)
#uni.columns = ["country", "university", "link"]

In [90]:
response = []
for df in unkown:
    unkown_ins = df["institution"].tolist()
    prompt_user = str(unkown_ins) + "\n\nhere is a list of institutions. Please return a list containing all the countries these institutions belong to. Please return in json format. If you do not know the country, please return unknown." 
    out = ChatGPT_request(prompt_user)
    response.append(out)
    print(out)

{
  "bnrist": "unknown",
  "boli technology co., ltd": "unknown",
  "booth school of business": "unknown",
  "bordeaux inp": "France",
  "boris ivanovic nvidia": "unknown",
  "boss zhipin": "unknown",
  "boston children's hospital": "United States",
  "brain team google research": "unknown",
  "brazilian federal police": "Brazil",
  "brendan dolangavitt": "unknown",
  "bristolmyers squibb": "unknown",
  "british antarctic survey": "United Kingdom",
  "british university vietnam": "Vietnam",
  "broad institute": "unknown",
  "broad institute of harvard and mit": "United States",
  "bruno kessler foundation fbk)": "unknown",
  "bsh hausgeräte gmbh": "Germany",
  "budapest university of technology": "Hungary",
  "bundeswehr university munich": "Germany",
  "burlington engineering laboratories": "unknown",
  "business university": "unknown",
  "byungkyu lee indiana university": "United States",
  "cadi ayyad university": "Morocco",
  "caeit": "unknown",
  "caidas": "unknown",
  "caiwen din

{
  "demacs": "unknown",
  "dept. of civil, construction, and environmental engineering": "unknown",
  "dept. of mechanical & aerospace engineering": "unknown",
  "desy zeuthen": "Germany",
  "dfki": "Germany",
  "dfki and rptu": "unknown",
  "dheevatsa mudigere §": "unknown",
  "di": "unknown",
  "dialpad canada inc": "Canada",
  "dialpad canada inc., ¶ royal bank of canada, $ salesforce research": "unknown",
  "diana liskovich": "unknown",
  "digital environment research institute": "unknown",
  "diro": "Unknown",
  "disi": "Italy",
  "dnv as": "unknown",
  "domaine universitaire de saintmartind'hères": "France",
  "donghua university": "China",
  "donguk kim": "unknown",
  "dp technology, ltd": "unknown",
  "dr. b.c. roy engineering college": "India",
  "dtu compute": "Denmark",
  "dtu electro": "Denmark",
  "du xiao man inc": "unknown",
  "duisburgessen university": "Germany",
  "duke university $ university of pennsylvania ‡ washington university in st. louis": "USA",
  "durham un

{
  "hannes kenngott": "unknown",
  "hanoi university of science and technology": "Vietnam",
  "harrisburg university of science and technology": "United States",
  "harrisen scells": "unknown",
  "harvard business school": "United States",
  "harvard cancer center": "United States",
  "harvard college": "United States",
  "harvard data science initiative": "United States",
  "harvard john a. paulson school of engineering and applied sciences": "United States",
  "harvard kennedy school": "United States",
  "harvard ophthalmology ai lab": "United States",
  "harvard university † emory university": "unknown",
  "harvard university ‡ stanford university": "unknown",
  "harvardmit": "United States",
  "harvardmit division of health sciences and technology": "United States",
  "harvardmit health sciences and technology": "United States",
  "hassan ii university of casablanca": "Morocco",
  "hatice gunes": "unknown",
  "health sciences and technology": "unknown",
  "health services research

ChatGPT ERROR
ChatGPT ERROR
{
  "karlsruhe university of applied sciences": "Germany",
  "karolinska institutet": "Sweden",
  "kavli energy nanoscience institute": "unknown",
  "kddi research": "Japan",
  "kddi research, inc": "Japan",
  "keck data science institute": "unknown",
  "keio university school of medicine": "Japan",
  "keji he": "unknown",
  "kempelen institute of intelligent technologies": "unknown",
  "kempner institute for study of natural & artificial intelligence": "unknown",
  "kennsaw state university": "unknown",
  "keyvan farahani": "unknown",
  "kharagpur ‡ presently in indian institute of technology": "India",
  "kharkiv national medical university": "Ukraine",
  "khoury college of computer science": "unknown",
  "kim jaechul graduate school of ai": "unknown",
  "king abdulaziz university": "Saudi Arabia",
  "king juan carlos university": "Spain",
  "kingstar technology inc": "unknown",
  "kioxia corporation": "unknown",
  "kirc": "unknown",
  "klaus maierhein": "

{
  "nanjing university of information science and technology": "China",
  "nanjing university of posts and telecommunication": "China",
  "nanjing university of science": "China",
  "nanjing university of science and technology nanjing": "China",
  "nanosystems institute": "unknown",
  "nanshan district": "unknown",
  "nantes université": "France",
  "nantes university": "France",
  "nanyang technological univerisity": "Singapore",
  "nanyang technological university ♠ salesforce ai northeastern university": "unknown",
  "nanyang technological university ntu)": "Singapore",
  "nanyang technological university zhiguo cao": "Singapore",
  "nanyang technology university bryan hooi": "Singapore",
  "nara": "Japan",
  "nara institute of science and technology": "Japan",
  "nari technology development limited company": "unknown",
  "narnia labs": "unknown",
  "nasa research": "United States",
  "nask national research institute": "Poland",
  "national and kapodistrian university of athens":

{
  "pla strategic support force information engineering university": "China",
  "pla strategiy support forces information engineering university": "China",
  "platform technologies": "unknown",
  "polish academy of sciences": "Poland",
  "politecnico di torino": "Italy",
  "politehnica university": "unknown",
  "poly corporation bellevue": "unknown",
  "polytechnic university of bari": "Italy",
  "polytechnic university of marche": "Italy",
  "pompeu fabra university": "Spain",
  "pontificia universidad católica de chile": "Chile",
  "pontifícia universidade católica do rio grande do sul av. ipiranga": "Brazil",
  "postdoctoral research associate": "unknown",
  "powerchina huadong engineering corporation limited": "China",
  "poznań university of technology": "Poland",
  "precision robotics hong kong) ltd": "Hong Kong",
  "proofpoint inc": "unknown",
  "psi": "unknown",
  "pskov state university": "Russia",
  "psl research univ": "France",
  "psyche ai inc": "unknown",
  "qatar comput

{
  "statistique et modélisation": "unknown",
  "stavanger university hospital": "Norway",
  "steklov institute of mathematics": "Russia",
  "stelios andrew stavroulakis": "unknown",
  "stellenbosch university": "South Africa",
  "stéphane lathuilière": "unknown",
  "stevens neuroimaging and informatics institute": "unknown",
  "stitchable neural networks": "unknown",
  "stochastic inc": "unknown",
  "subdirectorate general of terrestrial systems": "unknown",
  "sun financial holding co., ltd": "unknown",
  "sun yatsen univeristy": "unknown",
  "suzhou institute of biomedical engineering and technology": "China",
  "suzhou institute of trade & commerce": "China",
  "swedish escience research centre": "Sweden",
  "swiss ai lab idsia & università della svizzera italiana": "Switzerland",
  "swiss finance institute": "Switzerland",
  "swiss finance institute sfi)": "Switzerland",
  "symbiosis institute of technology": "India",
  "symbiosis international university siu)": "India",
  "system

{
   "university of guadalajara": "Mexico",
   "university of hagen": "Germany",
   "university of hawai'i at manoa": "United States",
   "university of hawaiʻi at mānoa": "United States",
   "university of hong kong ♣ university of cambridge": "Hong Kong",
   "university of hong kong https": "Hong Kong",
   "university of hong kong pokfulam": "Hong Kong",
   "university of hongkong": "Hong Kong",
   "university of hradec kralove": "Czech Republic",
   "university of information engineering": "unknown",
   "university of information technology": "unknown",
   "university of kaiserslauternlandau rptu)": "Germany",
   "university of kashan": "Iran",
   "university of koblenz": "Germany",
   "university of kwazulunatal": "South Africa",
   "university of la rochelle": "France",
   "university of lorraine": "France",
   "university of lübeck": "Germany",
   "university of lugano": "Switzerland",
   "university of manouba": "Tunisia",
   "university of mashhad": "Iran",
   "university of me

In [94]:
tmp = response[0]

In [92]:
#with open('/Users/zhangran/Documents/GitHub/Quaterly-Arxiv/data/unknown_chatGPT.json', 'w') as fp:
#    json.dump(response, fp)

In [113]:
lst = []
for tmp in response:
    if "ChatGPT ERROR" in tmp:
        pass
    else:
        for i in tmp[4:-3].split(",\n "):
            split = i.replace('"', '').split(": ")
            lst.append(split)

In [116]:
pd.DataFrame(lst, columns = ["institution", "country"]).to_csv("unknow_chatgpt_result.csv", index = False)

In [318]:
# graph
map_df = pd.read_csv("../code/affiliation - affiliation.csv")

In [319]:
map_df = map_df[map_df.iloc[:, -6:-1].isna().sum(axis = 1)<=4]

In [320]:
map_df.columns

Index(['id', 'institution', 'institution.1', 'loc1', 'loc2', 'loc3', 'loc4',
       'unknown ', 'combi'],
      dtype='object')

In [321]:
map_df = map_df[(map_df["unknown "] != "Unknown") & (map_df["unknown "] != "unknown")]

In [322]:
map_df["id"].nunique()

7479

In [323]:
map_df["combi"] = map_df.apply(lambda x: x.iloc[-6:-1].values, axis = 1)

In [324]:
import numpy as np
from collections import Counter
def clean(lst):
    try:
        return np.unique([i for i in lst if (isinstance(i, str)) and (i != "none")])
    except:
        return np.NAN

In [325]:
map_df["combi"] = map_df["combi"].apply(lambda x: clean(x))

In [326]:
map_df["region"] = map_df["loc2"]

In [327]:
geo = pd.read_csv("../code/geo.csv")

In [328]:
geo["name"] = [n.lower() for n in geo["name"]]

In [329]:
geo = geo.set_index("name")

In [330]:
for idx, row in map_df.iterrows():
    if isinstance(row["region"], float):
        try:
            name = row["combi"][0].lower()
        except:
            name = "missing"
        if (name == "usa") or (name == "united states") or (name == "us"):
            map_df.loc[idx, "region"] = "US"
        elif (name == "uk") or (name == "united kingdom"):
            map_df.loc[idx, "region"] = "GB"
        elif (name == "china"):
            map_df.loc[idx, "region"] = "CN"
        elif (name == "spain"):
            map_df.loc[idx, "region"] = "ES"
        elif (name == "france"):
            map_df.loc[idx, "region"] = "FR"
        elif (name == "germany"):
            map_df.loc[idx, "region"] = "DE"
        elif (name == "italy"):
            map_df.loc[idx, "region"] = "IT"
        elif (name == "japan"):
            map_df.loc[idx, "region"] = "JP"
        elif (name == "uae"):
            map_df.loc[idx, "region"] = "AE"
        else:
            try:
                map_df.loc[idx, "region"] = geo.loc[name]["country"]        
            except:
                if len(name) == 2:
                    map_df.loc[idx, "region"] = row["combi"][0].upper()
                else:
                    print(name)

telaviv
country not available
missing
telaviv
macao
macao
macao
macao
country
telaviv
country not available
missing
missing
missing
telaviv
isreal
telaviv
inc
polytechnique montréal
mines albi
telaviv
country not available
missing
country not available
missing
telaviv
telaviv
missing
telaviv
missing
missing
missing
country not available
polytechnique montréal
telaviv
country not available
missing
telaviv
missing
europe
missing
polytechnique montréal
country
telaviv
missing
polytechnique montréal
isreal


In [337]:
merged = pd.merge(map_df, geo, left_on = "region", right_on = "country", how = "left").reset_index()

In [341]:
merged["pro_country_perpaper"] = merged.groupby(["id", "region"], as_index = False).region.transform("count")

In [343]:
merged["total_ins_perpaper"] = merged.groupby(["id"], as_index = False).region.transform("count")

In [346]:
merged["agg_region"] = merged["pro_country_perpaper"]/merged["total_ins_perpaper"]

In [349]:
merged.to_csv("world_map_data.csv", index = False)